In [ ]:
# import pandas as pd
# import numpy as np
# from postgresql_conn import engine
# from pandasql import sqldf
# import operator
# from operator import itemgetter
# from IPython.display import display

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
# from sklearn.cross_validation import train_test_split, cross_val_score
# from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_curve, roc_auc_score
# from sklearn.feature_selection import RFE

# import matplotlib.pyplot as plt
# import seaborn as sns
# %matplotlib inline

# titlefont = {'fontname':'Times New Roman', 'fontweight':'bold', 'fontsize':18}
# labelfont = {'fontname':'Times New Roman', 'fontsize':16}

#####<font color='navy'>Functions</font>

In [ ]:
#get KNearestClassifier with maximum accuracy
def max_knn(X_train, y_train, X_test, y_test, n=20):
    """
    where n = the number of nearest neighbors (iterations)
        X_train = X/features training set
        y_train = y/response training set
        X_test = X/features test set
        y_test = y/response test set
    
    returns knum (int) = nearest neighbor index beginning 1 to n (e.g. 11)
            kval (float) = nearest neighbor accuracy value (e.g. 0.6745)
    """
    
    k_list = []
    for i in range(1, n+1):
        temp = []
        model = KNeighborsClassifier(n_neighbors=i).fit(X_train, y_train)
        temp.append("KNN %s:" % (i))
        temp.append("%.4f" % (accuracy_score(y_test, model.predict(X_test))))
        k_list.append(temp)
    k_dict = dict(k_list)
    
    knn = int(max(k_dict.iteritems(), key=itemgetter(1))[0].split()[1].split(':')[0]) #returns knn number
    kvalue = float(max(k_dict.iteritems(), key=itemgetter(1))[1])                 #returns knn value
    
    return knn, kvalue

#calculate metrics (accuracy, precision, recall, fscore)
def metrics(X_train, y_train, X_test, y_test, classifiers=[KNeighborsClassifier(n_neighbors=20), LogisticRegression(random_state=0), SVC(probability=True), GaussianNB(), DecisionTreeClassifier(), RandomForestClassifier(), ExtraTreesClassifier(n_estimators=25, max_depth=9, random_state=0)]):
    """
    where classifiers = list of classifier(s)
        X_train = X/features training set
        y_train = y/response training set
        X_test = X/features test set
        y_test = y/response test set
        
    returns list of lists (classifier, accuracy, precision, recall, f_score, auc)            
    """
    
    metrics = []
    for i in range(len(classifiers)):
        temp = []
        models[i] = classifiers[i].fit(X_train, y_train)
        temp.append(titles[i])
        temp.append("%.4f" % (accuracy_score(y_test, models[i].predict(X_test))))
        temp.append("%.4f" % (precision_recall_fscore_support(y_test, models[i].predict(X_test))[0][1]))
        temp.append("%.4f" % (precision_recall_fscore_support(y_test, models[i].predict(X_test))[1][1]))
        temp.append("%.4f" % (precision_recall_fscore_support(y_test, models[i].predict(X_test))[2][1]))
        temp.append("%.4f" % (roc_auc_score(y_test, [x[1] for x in models[i].predict_proba(X_test)])))
        metrics.append(temp)
        
    return metrics

def baseline(y_true):
    """
    baseline function which predicts true everytime
    """
    return [1 for y in range(len(y_true))]

In [ ]:
#brute force method of determining best n_estimator for ExtraTreesClassifier
from sklearn.ensemble import ExtraTreesClassifier

trees = []

for i in range(10, 26): #can be changed
    temp = []
    ex = ExtraTreesClassifier(n_estimators=i, random_state=1).fit(X_train, y_train)
    temp.append('Extra Tree %s' % str(i))
    temp.append("%.4f" % (accuracy_score(y_test, ex.predict(X_test))))
    temp.append("%.4f" % (precision_recall_fscore_support(y_test, ex.predict(X_test))[0][1]))
    temp.append("%.4f" % (precision_recall_fscore_support(y_test, ex.predict(X_test))[1][1]))
    temp.append("%.4f" % (precision_recall_fscore_support(y_test, ex.predict(X_test))[2][1]))
    temp.append("%.4f" % (roc_auc_score(y_test, [x[1] for x in ex.predict_proba(X_test)])))
    trees.append(temp)

pd.DataFrame(trees, columns=['Classifier', 'Accuracy', 'Precision', 'Recall', 'F_Score', 'AUC'])

In [ ]:
#brute force method of determining max_depth for ExtraTree Classifier

from sklearn.ensemble import ExtraTreesClassifier

trees = []

for i in range(21, 22): #can be changed
    for j in range(12, 13): #can be changed
        temp = []
        ex = ExtraTreesClassifier(n_estimators=i, max_depth=j, random_state=0).fit(X_train, y_train)
        temp.append('Extra Tree %s' % str(i))
        temp.append('Depth %s' % str(j))
        temp.append("%.4f" % (accuracy_score(y_test, ex.predict(X_test))))
        temp.append("%.4f" % (precision_recall_fscore_support(y_test, ex.predict(X_test))[0][1]))
        temp.append("%.4f" % (precision_recall_fscore_support(y_test, ex.predict(X_test))[1][1]))
        temp.append("%.4f" % (precision_recall_fscore_support(y_test, ex.predict(X_test))[2][1]))
        temp.append("%.4f" % (roc_auc_score(y_test, [x[1] for x in ex.predict_proba(X_test)])))
        trees.append(temp)
ex_features = sorted(zip(X.columns, ex.feature_importances_), key=operator.itemgetter(1), reverse=True)
#sorted(zipped, key=operator.itemgetter(1))
a = pd.DataFrame(trees, columns=['n_estimator', 'max_depth', 'Accuracy', 'Precision', 'Recall', 'F_Score', 'AUC'])
a   

####<font color='navy'>Case 1: Raw Hypertension Data (raw_hypertension_fullset)</font>
####<font color='navy'>Case 2: Binary Hypertension Data (r_converted_data)</font>
####<font color='navy'>Case 3: Raw Hypertension Data with recursive feature elimination (raw_hypertension_rfe_less)</font>
####<font color='navy'>Case 4: Binary Hypertension Data with recursive feature elimination (r_converted_data_less)</font>

In [ ]:
#Load dataset
bp = pd.read_sql('raw_hypertension_fullset', engine)
print bp.shape
bp = bp.dropna()
print bp.shape

#Create training and test sets
y = bp['highbloodstatus']
X = bp.drop('highbloodstatus', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.20, random_state=0)

#Check baseline model
baseline_li = []
base_temp = []  
base_temp.append('Baseline')
base_temp.append("%.4f" % (accuracy_score(y_test, baseline(y_test))))
base_temp.append("%.4f" % (precision_recall_fscore_support(y_test, baseline(y_test))[0][1]))
base_temp.append("%.4f" % (precision_recall_fscore_support(y_test, baseline(y_test))[1][1]))
base_temp.append("%.4f" % (precision_recall_fscore_support(y_test, baseline(y_test))[2][1]))
base_temp.append("%.4f" % (roc_auc_score(y_test, [1 for x in range(len(y_test))])))
baseline_li.append(base_temp)

baseline_df = pd.DataFrame(baseline_li, columns=['Classifier', 'Accuracy', 'Precision', 'Recall', 'F_Score', 'AUC'])
display(baseline_df)

baseline_fpr, baseline_tpr, _ = roc_curve(y_test, [1 for x in range(len(y_test))])
plt.figure(figsize=(10, 5))
plt.plot(baseline_fpr, baseline_tpr)
plt.xlabel('False Positive Rate', **labelfont)
plt.ylabel('True Positive Rate', **labelfont)
plt.title('Baseline ROC Curve', **titlefont)

#Build models
knum, kval = max_knn(X_train, y_train, X_test, y_test, 18)

classifiers = [KNeighborsClassifier(knum), LogisticRegression(random_state=0), SVC(probability=True), GaussianNB(), BernoulliNB(), MultinomialNB(), DecisionTreeClassifier(random_state=0), RandomForestClassifier(random_state=0), ExtraTreesClassifier(n_estimators=21, max_depth=12, random_state=0)]
models = ['knn', 'log', 'svc', 'gaussian', 'bernoulli', 'multinomial', 'tree', 'forest', 'extrees']
titles = ['KNN', 'Log', 'SVC', 'Gaussian', 'Bernoulli', 'Multinomial', 'Decision Tree', 'Random Forest', 'Extra Trees']

#Calculate metrics
scores = metrics(X_train, y_train, X_test, y_test, classifiers)
metrics_df = pd.DataFrame(metrics(X_train, y_train, X_test, y_test, classifiers), columns=['Classifier', 'Accuracy', 'Precision', 'Recall', 'F_Score', 'AUC'])
display(metrics_df)

fpr_list, tpr_list = [], []
for i in range(len(classifiers)):
    class_ = classifiers[i].fit(X_train, y_train)
    fpr, tpr, _ = roc_curve(y_test, [x[1] for x in class_.predict_proba(X_test)])
    fpr_list.append(fpr)
    tpr_list.append(tpr)

#Create plots
colors = ['Red', 'Yellow', 'Blue', 'Purple', 'Teal', 'Cyan', 'Olive', 'Green', 'Maroon']

plt.figure(figsize=(15, 10)) #single plot
for i in range(len(fpr_list)):
    plt.plot(fpr_list[i], tpr_list[i], color=colors[i], label=titles[i])
    plt.xlabel('False Positive Rate', **labelfont)
    plt.ylabel('True Positive Rate', **labelfont)
    plt.title('ROC Curves', **titlefont)
    leg = plt.legend(loc=4, frameon=True, fontsize=12)
    leg.get_frame().set_edgecolor('dimgrey')
    
plt.figure(figsize=(12,15)) #one plot for each model
with sns.axes_style("darkgrid"):
    sns.set_style("darkgrid")
    for i in range(len(fpr_list)):
        plt.subplot(5,2,i+1)
        plt.plot(fpr_list[i], tpr_list[i], color=colors[i], label="AUC: %.4f" % float(scores[i][5]))
        plt.xlabel('False Positive Rate', **labelfont)
        if i%2==0:
            plt.ylabel('True Positive Rate', **labelfont)
        plt.title(titles[i], **titlefont)
        leg = plt.legend(loc=2, frameon=True, fontsize=12)
        leg.get_frame().set_edgecolor('dimgrey')
    plt.tight_layout(pad=0.4, w_pad=0.2, h_pad=2.0)

In [ ]:
#create baseline + classifiers model for dataset 1
merge = """
SELECT Classifier, 'Model 1' AS Model, 'Raw Full (43)' AS Dataset, Accuracy, Precision, Recall, F_Score, AUC
FROM baseline_df 
UNION 
SELECT Classifier, 'Model 1', 'Raw Full (43)', Accuracy, Precision, Recall, F_Score, AUC
FROM metrics_df
"""

acc_rec_df_1 = sqldf(merge, globals())
acc_rec_df_1.head()

In [ ]:
#create baseline + classifiers model for dataset 2
merge = """
SELECT Classifier, 'Model 2' AS Model, 'Converted Full (42)' AS Dataset, Accuracy, Precision, Recall, F_Score, AUC
FROM baseline_df 
UNION 
SELECT Classifier, 'Model 2', 'Converted Full (42)', Accuracy, Precision, Recall, F_Score, AUC
FROM metrics_df
"""

acc_rec_df_2 = sqldf(merge, globals())
acc_rec_df_2.head()

In [ ]:
#create baseline + classifiers model for dataset 3
merge = """
SELECT Classifier, 'Model 3' AS Model, 'Raw Less (21)' AS Dataset, Accuracy, Precision, Recall, F_Score, AUC
FROM baseline_df 
UNION 
SELECT Classifier, 'Model 3', 'Raw Less (21)', Accuracy, Precision, Recall, F_Score, AUC
FROM metrics_df
"""

acc_rec_df_3 = sqldf(merge, globals())
acc_rec_df_3.head()

In [ ]:
#create baseline + classifiers model for dataset 4
merge = """
SELECT Classifier, 'Model 4' AS Model, 'Converted Less (20)' AS Dataset, Accuracy, Precision, Recall, F_Score, AUC
FROM baseline_df 
UNION 
SELECT Classifier, 'Model 4', 'Converted Less (20)', Accuracy, Precision, Recall, F_Score, AUC
FROM metrics_df
"""

acc_rec_df_4 = sqldf(merge, globals())
acc_rec_df_4.head()

In [ ]:
union = """
SELECT Classifier, Model, Dataset, Accuracy, Precision, Recall, F_Score, AUC
FROM acc_rec_df_1
UNION
SELECT Classifier, Model, Dataset, Accuracy, Precision, Recall, F_Score, AUC
FROM acc_rec_df_2
UNION
SELECT Classifier, Model, Dataset, Accuracy, Precision, Recall, F_Score, AUC
FROM acc_rec_df_3
UNION
SELECT Classifier, Model, Dataset, Accuracy, Precision, Recall, F_Score, AUC
FROM acc_rec_df_4
ORDER BY Model, Classifier
"""

s = sqldf(union, globals())
s.head()

In [ ]:
#create csv file for d3
bar = """
SELECT Classifier
    , SUM(CASE WHEN Model = 'Model 1' THEN Recall END) AS Model_1_Raw
    , SUM(CASE WHEN Model = 'Model 2' THEN Recall END) AS Model_2_Binary
    , SUM(CASE WHEN Model = 'Model 3' THEN Recall END) AS Model_3_Raw_RFE
    , SUM(CASE WHEN Model = 'Model 4' THEN Recall END) AS Model_4_Binary_RFE
FROM s
GROUP BY Classifier
"""

sqldf(bar, globals()).to_csv('pivot_recall.csv', index=False)

#####<font color='navy'>Cross validation</font>

In [ ]:
#[KNeighborsClassifier(knum), LogisticRegression(), SVC(probability=True), GaussianNB(), BernoulliNB(), MultinomialNB(), DecisionTreeClassifier(), RandomForestClassifier()]
models = [BernoulliNB(), ExtraTreesClassifier(n_estimators=19, max_depth=8, random_state=0), LogisticRegression(random_state=0), GaussianNB()]
name = ['Binomial', 'Extra Trees', 'Log', 'Gaussian']
scoring = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
crossval = []

for i in range(len(models)):
    for j in range(len(scoring)):
        tempscore = []    
        tempscore.append(name[i])
        tempscore.extend(cross_val_score(models[i], X.values, y.values, scoring=scoring[j], cv=5)) 
        crossval.append(tempscore)
crossval_df = pd.DataFrame(crossval, columns=['Classifier', 'Accuracy', 'Precision', 'Recall', 'F_Score', 'AUC'])

cv_mean = """
            SELECT Classifier
                , CAST(AVG(Accuracy) AS DECIMAL(10,4)) AS Accuracy
                , CAST(AVG(Precision) AS DECIMAL(10,4)) AS Precision
                , CAST(AVG(Recall) AS DECIMAL(10,4)) AS Recall
                , CAST(AVG(F_Score) AS DECIMAL(10,4)) AS F_Score
                , CAST(AVG(AUC) AS DECIMAL(10,4)) AS AUC
            FROM crossval_df
            GROUP BY Classifier
        """

sqldf(cv_mean, globals())

In [ ]:
#check for the important features (feature importance)
rfe_class = [LogisticRegression(random_state=0), BernoulliNB(), MultinomialNB()]
important = []
for i in range(len(rfe_class)):
    temp = []
    models[i] = RFE(rfe_class[i])
    models[i] = models[i].fit(X_train, y_train)
    important.append(zip(X_train.columns, models[i].support_))
    #print models[i].n_features_

In [ ]:
l = pd.DataFrame(important[0], columns=['Feature', 'Desired'])
b = pd.DataFrame(important[1], columns=['Feature', 'Desired'])
m = pd.DataFrame(important[2], columns=['Feature', 'Desired'])

In [ ]:
lbm = """
SELECT Feature, Log, Bernoulli, Multinomial
, Log+Bernoulli+Multinomial AS Desired
FROM(
SELECT l.Feature
, CASE WHEN l.Desired THEN 1 ELSE 0 END AS Log
, CASE WHEN b.Desired THEN 1 ELSE 0 END AS Bernoulli
, CASE WHEN m.Desired THEN 1 ELSE 0 END AS Multinomial
FROM l JOIN
b ON l.Feature = b.Feature JOIN
m ON l.Feature = m.Feature
) AS A
"""

#rfe_conv = sqldf(lbm, globals())#.to_csv('RFE_Converted.csv')
#rfe_raw = sqldf(lbm, globals())
sqldf(lbm, globals())

In [ ]:
heat = """
SELECT CASE WHEN Raw_Desired IS NULL THEN 0 ELSE Raw_Desired END AS Raw_Desired
    , CASE WHEN Conv_Desired IS NULL THEN 0 ELSE Conv_Desired END AS Conv_Desired
FROM(
    SELECT rfe_conv.Feature AS Feature, rfe_raw.Desired AS Raw_Desired, rfe_conv.Desired AS Conv_Desired
    FROM rfe_conv LEFT JOIN
        rfe_raw ON rfe_conv.Feature = rfe_raw.Feature
    UNION 
    SELECT rfe_raw.Feature, rfe_raw.Desired, rfe_conv.Desired
    FROM rfe_raw LEFT JOIN
        rfe_conv ON rfe_conv.Feature = rfe_raw.Feature
) AS A
"""
heatmap = sqldf(heat, globals())
type(heatmap)

# heat = """
# SELECT DISTINCT Feature
# FROM(
# SELECT rfe_raw.Feature AS Feature, 'RAW' AS Model, rfe_raw.Desired AS Desired
# FROM rfe_raw
# UNION
# SELECT rfe_conv.Feature, 'CONVERTED', rfe_conv.Desired
# FROM rfe_conv
# ) A
# ORDER BY Feature
# """
# print [x[0].encode('utf8') for x in sqldf(heat, globals()).values.tolist()]#.to_csv('heat_unpivot.csv', index=False)

In [ ]:
#heatmap_pivot = pd.DataFrame()
heatmap = np.array(heatmap)
heatmap_pivot = []
temp = []
temp2 = []
for x in heatmap:

    temp.append(x[0])
    temp2.append(x[1])
heatmap_pivot.append(temp)
heatmap_pivot.append(temp2)

In [ ]:
heatmap_1 = [x[:24] for x in heatmap_pivot]
heatmap_2 = [x[24:] for x in heatmap_pivot]
xlabel_1 = ylabel[:24]
xlabel_2 = ylabel[24:]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 1))
sns.heatmap(heatmap_1, yticklabels=['Raw', 'Converted'], xticklabels=xlabel_1, linewidths=.5, cbar=False, ax=ax)
ax.tick_params(axis='y', labelsize=18)
ax.tick_params(axis='x', labelsize=14)
plt.xticks(rotation=45, ha='right')


fig2, ax2 = plt.subplots(1, 1, figsize=(10, 1))
sns.heatmap(heatmap_2, yticklabels=['Raw', 'Converted'], xticklabels=xlabel_2, linewidths=.5, cbar=False, ax=ax2)
ax2.tick_params(axis='y', labelsize=18)
ax2.tick_params(axis='x', labelsize=14)
# fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 2))
# sns.heatmap(heatmap_1, yticklabels=['Raw', 'Converted'], xticklabels=xlabel_1, linewidths=.5, cbar=False, ax=ax1)
# sns.heatmap(heatmap_2, yticklabels=['Raw', 'Converted'], xticklabels=xlabel_2, linewidths=.5, cbar=False, ax=ax2)
# ax1.tick_params(axis='x', labelsize=14)
# ax1.tick_params(axis='y', labelsize=18)
# ax2.tick_params(axis='x', labelsize=14)
# ax2.tick_params(axis='y', labelsize=18)
plt.xticks(rotation=45, ha='right')

plt.savefig('Important_Features_Heatmap.jpeg')

In [ ]:
conv = pd.read_csv('RFE_Converted.csv')
len(conv[conv['Desired'] > 0])

In [ ]:
raw = pd.read_csv('RFE_Raw.csv')
len(raw[raw['Desired'] > 0])

In [ ]:
pd.read_csv('acc_and_recall.csv')